In [15]:
import numpy as np 
import os
from PIL import Image
from keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.layers import *
from keras.models import Model, load_model

In [16]:
img_data=[]
img_labels=[]

labels={'paper':0,'rock':1,'scissors':2}

dirPath='image_data'

for folder in os.listdir(dirPath):
    folder_path=os.path.join(dirPath,folder)
    print(folder_path)
    for pic in os.listdir(folder_path):
        pic_path=os.path.join(folder_path,pic)
        pic=Image.open(pic_path)
        pic=np.asarray(pic)
        
        img_data.append(pic)
        img_labels.append(labels[folder])

XTrain=np.asarray(img_data)
YTrain=np.asarray(img_labels)

image_data\paper
image_data\rock
image_data\scissors


In [17]:
YTrain=to_categorical(YTrain)

In [18]:
model=ResNet50(include_top=False, weights='imagenet',input_shape=(200,300,3))

In [19]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 206, 306, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 100, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [20]:
av1=GlobalAveragePooling2D()(model.output)
fc1=Dense(256,activation='relu')(av1)
d1=Dropout(0.5)(fc1)
fc2=Dense(3,activation='softmax')(d1)

model_new=Model(inputs=model.input, outputs=fc2)
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 206, 306, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 100, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [21]:
adam=Adam(lr=0.00003)
model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])

In [22]:
for ix in range(len(model_new.layers)):
    print(ix,model_new.layers[ix])

0 <keras.engine.input_layer.InputLayer object at 0x00000159DDDAD610>
1 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000159A28A4370>
2 <keras.layers.convolutional.conv2d.Conv2D object at 0x00000159A28D5B80>
3 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000015B90069DF0>
4 <keras.layers.core.activation.Activation object at 0x00000159DB6B8F40>
5 <keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000159A286BAF0>
6 <keras.layers.pooling.max_pooling2d.MaxPooling2D object at 0x0000015B90407BB0>
7 <keras.layers.convolutional.conv2d.Conv2D object at 0x00000159DDE4BAC0>
8 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000159DDE4B190>
9 <keras.layers.core.activation.Activation object at 0x00000159DDE46A90>
10 <keras.layers.convolutional.conv2d.Conv2D object at 0x00000159DDE436A0>
11 <keras.layers.normalization.batch_normalization.BatchNormalization object at 0x0000015B90409FD0>
12 <keras.lay

In [23]:
for ix in range(169):
    model_new.layers[ix].trainable=False 

model_new.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model_new.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 200, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 206, 306, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 100, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [24]:
hist=model_new.fit(XTrain,YTrain,shuffle=True,batch_size=16,epochs=5,validation_split=0.20)

Epoch 1/5
60/60 [==============================] - 8s 88ms/step - loss: 0.5609 - accuracy: 0.7719 - val_loss: 0.4186 - val_accuracy: 0.9917
Epoch 2/5
60/60 [==============================] - 4s 72ms/step - loss: 0.1269 - accuracy: 0.9885 - val_loss: 0.2405 - val_accuracy: 0.9875
Epoch 3/5
60/60 [==============================] - 4s 72ms/step - loss: 0.0609 - accuracy: 0.9969 - val_loss: 0.1730 - val_accuracy: 0.9917
Epoch 4/5
60/60 [==============================] - 4s 72ms/step - loss: 0.0355 - accuracy: 1.0000 - val_loss: 0.1045 - val_accuracy: 0.9917
Epoch 5/5
60/60 [==============================] - 4s 72ms/step - loss: 0.0231 - accuracy: 1.0000 - val_loss: 0.0555 - val_accuracy: 1.0000


In [25]:
model_new.save("ClassificationModel.h5")

In [26]:
model=load_model('ClassificationModel.h5')

In [27]:
predictions=model.predict(XTrain)
predicted_classes = np.argmax(predictions, axis=1)

38/38 [==============================] - 5s 107ms/step


In [28]:
print(predicted_classes)

[0 0 0 ... 2 2 2]
